# Import libraries

In [1]:
import pandas as pd
import numpy as np
import copy, random, math
import ast
import folium

from geopy.distance import geodesic
from folium.plugins import MarkerCluster

# Loading data & Data preparation

In [2]:
distances = pd.read_csv('distances.csv')
establishments = pd.read_csv('establishments.csv')

# shift the column names to the left
distances.columns = pd.Index(list(distances.columns[1:]) + ['p_1000'])

very_small_establishments = establishments.iloc[0:21,:]
very_small_distances = distances.iloc[0:21,0:21]

small_establishments = establishments.iloc[0:101,:]
small_distances = distances.iloc[0:101,0:101]

medium_establishments = establishments.iloc[0:501,:]
medium_distances = distances.iloc[0:501,0:501]

large_establishments = establishments
large_distances = distances

# Problem definition

In [3]:
# convert the DataFrame to a dictionary
traveling_times = {}
for i in range(len(large_distances)):
    traveling_times[i] = large_distances.iloc[i].tolist()

num_establishments = len(traveling_times)-1
vehicles = int(num_establishments*0.1)
inspection_times = large_establishments[['Inspection Time']]
open_hours = large_establishments[['Opening Hours']]


def generate_random_solution():
    return np.random.randint(1, vehicles + 1, num_establishments)

def objective_function(chromosome, traveling_times, inspection_times, open_hours):
    total_travel_time = 0
    total_inspection_time = 0
    total_time_per_route = [0 for i in range(max(chromosome))]
    j = -1

    # Initialize an empty list to store the routes
    routes = [[] for i in range(max(chromosome))]
    best_routes = []

    # Assign each establishment to a route
    for i in range(len(chromosome)):
        routes[chromosome[i]-1].append(i+1)

    # Calculate the total travel time and inspection time for all routes
    for route in routes:
        if len(route) == 0:
            continue
        j += 1
        route_travel_time = 0
        route_inspection_time = 0
        current_time = 0
        # Add the departure/arrival establishment to the beginning and end of the route
        route = [0] + route + [0]
        
        # Use a greedy search algorithm to find the optimal path for the current route
        route = greedy_search(route, heuristic_distances)
        best_routes.append(route)
        
        start_time = ast.literal_eval(open_hours.iloc[route[1]][0]).index(1)  # Inspection start time of first establishment in route
        #print('start time: {}'.format(start_time))

        for i in range(1, len(route)):
            # Calculate traveling time between establishments
            current_establishment = route[i-1]
            next_establishment = route[i]
            travel_time = traveling_times[current_establishment][next_establishment]
            #route_travel_time += traveling_times[current_establishment][next_establishment]
            
            # Calculate inspection time and waiting time based on establishment's schedule
            current_inspection_time = inspection_times.iloc[next_establishment][0]*60 # Convert minutes to seconds
            current_open_hours = ast.literal_eval(open_hours.iloc[next_establishment][0])
            
            # Calculate waiting time if establishment is closed
            if i == 1:
                waiting_time = 0
            else:
                current_hour = int((current_time + travel_time + (start_time*3600))/3600)
                #print('Current hour: {}'.format(current_hour))
                while current_open_hours[current_hour % 24] == 0:
                    current_hour += 1
                waiting_time = (current_hour * 3600) - (current_time + travel_time + (start_time*3600))
                #print('({} * 3600) - ({} + {} + ({}*3600))'.format(current_hour, current_time, travel_time, start_time))
                
            # Add inspection time and waiting time to current time
            if i > 1:
                current_time += max(waiting_time, 0)
            current_time += travel_time
            current_time += current_inspection_time
            
            #print('Iteration {} ---> current establishment: {}, next establishment: {}, inspection time: {}, open hours array: {}, travel time: {}, waiting time: {}, current time: {}'.format(i, current_establishment, next_establishment, current_inspection_time, current_open_hours, travel_time, waiting_time, current_time))
            # Reset waiting time
            waiting_time = 0
            
            # Add inspection time and traveling time to route times
            #route_inspection_time += current_inspection_time
            route_travel_time = 0

        # Add total time for current route to total time per route
        total_time_per_route[j] = current_time
        #print('total time per route ----> {}'.format(total_time_per_route))

    # Add total time for all routes to total travel time
    total_travel_time += sum(total_time_per_route)
    
    return -total_travel_time, best_routes # Minimize the sum of inspection and travel time

def objective_function_a_star(chromosome, traveling_times, inspection_times, open_hours):
    
    total_travel_time = 0
    l = 1
    
    # Initialize an empty list to store the routes
    routes = [[] for i in range(max(chromosome))]
    best_routes = []
    
    # Assign each establishment to a route
    for i in range(len(chromosome)):
        routes[chromosome[i]-1].append(i+1)
    
    # Calculate the total travel time for all routes
    for route in routes:
        if len(route) == 0:
            continue
        j = 1
        current_time = 0
        
        # Create a set with all establishments in the route
        route_establishments = set(route)
        route_len = len(route_establishments)
        
        # Add the depot to the beginning of the route
        route = [0]
        
        # Initialize the route travel time as zero
        route_travel_time = 0
        
        while route_establishments:
            # Calculate the A* algorithm for the next establishment
            current_establishment = route[-1]
            next_establishment = None
            best_score = float('inf')
            for e in route_establishments:
                i = 1
                # Calculate the total time (travel time, inspection time, waiting time, heuristic value) for the next establishment
                travel_time = traveling_times[route[-1]][e]
                inspection_time = inspection_times.iloc[e][0] * 60
                current_open_hours = ast.literal_eval(open_hours.iloc[e][0])
                if len(route_establishments) == route_len:
                    start_time = current_open_hours.index(1)
                i += 1
                # Calculate waiting time if establishment is closed
                if len(route) == 1:
                    waiting_time = 0
                else:
                    current_hour = int((current_time + travel_time + (start_time*3600))/3600)
                    while current_open_hours[current_hour % 24] == 0:
                        current_hour += 1
                    waiting_time = (current_hour * 3600) - (current_time + travel_time + (start_time*3600))
                
                # Add inspection time, waiting time, and travel time to the current time
                time_to_next = max(waiting_time, 0) + inspection_time + travel_time
                total_time = current_time + time_to_next
                heuristic_value = heuristic_distances[e][0]
                score = total_time + heuristic_value
                
                # If the current establishment has the smallest score, update next_establishment
                if score < best_score:
                    '''
                    best_waiting_time = max(waiting_time, 0)
                    best_open_hours = current_open_hours
                    best_inspection_time = inspection_time
                    if len(route) == 1:
                        best_start_time = start_time
                    else:
                        best_current_hour = current_hour
                    best_current_time = total_time
                    best_travel_time = travel_time
                    '''
                    time_to_next_establishment = time_to_next
                    next_establishment = e
                    best_score = score
            '''
            if len(route) > 1:
                print('Current hour: {}'.format(best_current_hour))
                print('({} * 3600) - ({} + {} + ({}*3600))'.format(best_current_hour, current_time, best_travel_time, best_start_time))
            else:
                print('start time: {}'.format(best_start_time))
            '''
            
            # Add next establishment to route and remove from set
            route_establishments.remove(next_establishment)
            route.append(next_establishment)
            
            #print('Update current time: {} + {}'.format(current_time, time_to_next_establishment))
            # Update the current time and route travel time
            current_time += time_to_next_establishment
            #route_travel_time += travel_time
            #print('Iteration {} ---> current establishment: {}, next establishment: {}, next inspection time: {}, open hours array: {}, travel time: {}, waiting time: {}, current time: {}'.format(j, current_establishment, next_establishment, best_inspection_time, best_open_hours, best_travel_time, best_waiting_time, best_current_time))
        
            j += 1
        # Add the depot to the end of the route
        route.append(0)
        best_routes.append(route)
        #print('Route: {}'.format(route))
        # Calculate the total time for the current route and add to total time per route
        total_time_per_route = current_time + traveling_times[route[-2]][0]
        #print('Traveling time back to the depot: {}'.format(traveling_times[route[-2]][0]))
        #print('Total travel time in route {}: {}'.format(l, total_time_per_route))
        total_travel_time += total_time_per_route
        l += 1
        
    return -total_travel_time, best_routes # Minimize the total travel time

def geodesic_time_matrix(establishments, speed_kmph):
    """
    Calculates the travel time matrix between all combinations of establishments based on their
    latitude and longitude coordinates, assuming a given speed in km/h.
    
    Parameters:
    establishments (pandas.DataFrame): A DataFrame containing the latitude and longitude coordinates of all establishments.
    speed_kmph (float): The speed of travel in km/h.
    
    Returns:
    pandas.DataFrame: A DataFrame containing the travel time matrix between all combinations of establishments in seconds.
    """
    # Create a 2D array of establishment coordinates
    coordinates = establishments[['Latitude', 'Longitude']].values
    
    # Initialize an empty time matrix with the same shape as the coordinates matrix
    time_matrix = pd.DataFrame(index=establishments.index, columns=establishments.index)
    
    # Calculate pairwise travel times between all establishments
    for i in range(len(coordinates)):
        for j in range(len(coordinates)):
            # Calculate the distance between establishments i and j using geodesic distance
            distance_km = geodesic(coordinates[i], coordinates[j]).km
            
            # Calculate the travel time in seconds using the speed in km/h
            travel_time_sec = (distance_km / speed_kmph) * 3600
            
            # Set the travel time in the time matrix
            time_matrix.iloc[i, j] = travel_time_sec
    
    return time_matrix

def greedy_search(route, heuristic):
    current_node = 0
    visited = [False] * (max(route) + 1)
    visited[current_node] = True
    path = [current_node]
    while len(path) < len(route):
        next_node = -1
        min_distance = float('inf')
        for i in range(len(route)):
            if not visited[route[i]]:
                distance = heuristic[route[i]][route[0]]
                if distance < min_distance:
                    next_node = route[i]
                    min_distance = distance
        visited[next_node] = True
        path.append(next_node)
        current_node = next_node
    path.pop() # remove last element (should be 0)
    path.append(0)
    return path

def display_routes(routes):
    # Define the map center and zoom level
    center = [41.160304, -8.602478]
    zoom = 10.4

    # Create a map object
    tile = 'OpenStreetMap'
    map = folium.Map(location=center, zoom_start=zoom, tiles=tile)

    # Define the color palette for the routes
    route_colors = ['#d62728','#3388ff','#33cc33','#ff9933','#800080','#ff3399','#808080','#f5f5dc','#32174d','#ffffff',
              '#5f9ea0','#d3d3d3','#add8e6','#00008b','#90ee90','#006400','#8b0000','#ff6666']
    marker_colors = ['red', 'blue', 'green', 'orange', 'purple', 'pink', 'gray', 'beige', 'darkpurple', 'white', 
                     'cadetblue', 'lightgray', 'lightblue', 'darkblue', 'lightgreen', 'darkgreen', 'darkred', 'lightred']

    # Create checkboxes for each route
    checkboxes = []
    for i, route in enumerate(routes):
        label = f"Route {i+1}"
        checkbox = folium.FeatureGroup(name=label, overlay=True, control=True)
        for k, j in enumerate(route[1:-1]):
            folium.Marker(
                location=(establishments.iloc[j]['Latitude'], establishments.iloc[j]['Longitude']),
                icon=folium.Icon(color=marker_colors[i % len(route_colors)]),
                tooltip=f"Visit order: {k+1}",
                popup=f"{establishments.iloc[j]['Address']}",
            ).add_to(checkbox)
        folium.PolyLine(
            locations=[(establishments.iloc[j]['Latitude'], establishments.iloc[j]['Longitude']) for j in route],
            color=route_colors[i % len(route_colors)],
            popup=label,
        ).add_to(checkbox)
        checkboxes.append(checkbox)
    
    # Add caution marker to the depot
    folium.Marker(location=center, icon=folium.Icon(icon='home', prefix='fa', color='black'), tooltip='Depot', 
                 popup=f"{establishments.iloc[0]['Address']}").add_to(map)
    
    # Add the checkboxes to the map
    for checkbox in checkboxes:
        checkbox.add_to(map)
    
    # Add a layer control to the map
    folium.LayerControl().add_to(map)
    
    # Display the map
    return map

In [4]:
heuristic_distances = geodesic_time_matrix(establishments, 70)

# Subtract the heuristic distance matrix from the travel time matrix
difference_matrix = heuristic_distances - distances

# Check if there are any positive values in the difference matrix
if (difference_matrix > 0).any().any():
    print('There is at least one travel time that is lower than the heuristic value!')
else:
    print('All travel times are greater than or equal to the heuristic value.')

All travel times are greater than or equal to the heuristic value.


# Genetic Algorithm

In [5]:
def midpoint_crossover(solution_1, solution_2):
    mid_index = math.trunc(len(solution_1) / 2)
    child_1 = np.append(solution_1[0:mid_index], solution_2[mid_index:]) 
    child_2 = np.append(solution_2[0:mid_index], solution_1[mid_index:])
    return child_1, child_2

def randompoint_crossover(solution_1, solution_2):
    length = len(solution_1)
    crossover_point = random.randint(1, length - 1)
    child_1 = np.concatenate([solution_1[:crossover_point], solution_2[crossover_point:]])
    child_2 = np.concatenate([solution_2[:crossover_point], solution_1[crossover_point:]])
    return child_1, child_2

def uniform_crossover(solution_1, solution_2):
    """
    Performs uniform crossover (UX) on two solutions.
    
    Args:
        solution_1 (list): The first solution.
        solution_2 (list): The second solution.
        
    Returns:
        Two new solutions created by randomly swapping genes between solution_1 and solution_2.
    """
    # Initialize children as copies of the parent solutions
    child_1 = solution_1.copy()
    child_2 = solution_2.copy()

    # Determine the genes to swap
    genes_to_swap = np.random.choice(len(solution_1), len(solution_1), replace=False)

    # Swap the genes between the two children
    for i in genes_to_swap:
        if child_1[i] != child_2[i]:
            child_1[i], child_2[i] = child_2[i], child_1[i]

    return child_1, child_2

def mutate_solution_1(solution):
    index_1 = np.random.randint(0, len(solution))
    index_2 = (index_1 + np.random.randint(0, len(solution))) % (len(solution) - 1) # Efficient way to generate a non-repeated index
    solution[index_1], solution[index_2] = solution[index_2], solution[index_1]
    return solution

def mutate_solution_2(solution):
    index = np.random.randint(0, len(solution))
    solution[index] = np.random.randint(1, vehicles + 1)
    return solution

def generate_population(population_size):
    solutions = []
    for i in range(population_size):
        solutions.append(generate_random_solution())
    return solutions

def print_population(population, objective_func):
    solutions = []
    for i in range(len(population)):
        print(f"Solution {i+1}: {population[i]}, {-objective_func(population[i], traveling_times, inspection_times, open_hours)[0]}")
        
def tournament_select(population, tournament_size, objective_func):
    population_copy = copy.deepcopy(population)
    best_solution = population_copy[0]
    best_score = objective_func(population_copy[0], traveling_times, inspection_times, open_hours)[0]
    for i in range(tournament_size):
        index = np.random.randint(0, len(population_copy))
        score = objective_func(population_copy[index], traveling_times, inspection_times, open_hours)[0]
        if score > best_score:
            best_score = score
            best_solution = population_copy[index]
        del population_copy[index]
    return best_solution

def get_greatest_fit(population, objective_func):
    best_solution = population[0]
    best_score, best_routes = objective_func(population[0], traveling_times, inspection_times, open_hours)
    for i in range(1, len(population)):
        score, routes = objective_func(population[i], traveling_times, inspection_times, open_hours)
        if score > best_score:
            best_score = score
            best_solution = population[i]
            best_routes = routes
    return best_solution, best_score, best_routes

def replace_least_fittest(population, offspring, objective_func):
    least_fittest_index = 0
    least_fittest_value = objective_func(population[0], traveling_times, inspection_times, open_hours)[0]
    for i in range(1, len(population)):
        score = objective_func(population[i], traveling_times, inspection_times, open_hours)[0]
        if score < least_fittest_value:
            least_fittest_value = score
            least_fittest_index = i
    population[least_fittest_index] = offspring

def roulette_select(population, objective_func):
    score_sum = sum([objective_func(solution, traveling_times, inspection_times, open_hours)[0] for solution in population])
    selection_probabilities = [objective_func(solution, traveling_times, inspection_times, open_hours)[0] / score_sum for solution in population]
    return population[np.random.choice(len(population), p=selection_probabilities)]

In [6]:
#Test Crossover
s1 = generate_random_solution()
s2 = generate_random_solution()
print('solution 1 -----> {}'.format(s1))
print('solution 2 -----> {}'.format(s2))
c1, c2 = midpoint_crossover(s1, s2)
print('child 1 -----> {}'.format(c1))
print('child 2 -----> {}'.format(c2))
c1, c2 = randompoint_crossover(s1, s2)
print('new child 1 -----> {}'.format(c1))
print('new child 2 -----> {}'.format(c2))
c1, c2 = uniform_crossover(s1, s2)
print('new child 1 ux -----> {}'.format(c1))
print('new child 2 ux -----> {}'.format(c2))
#Test Mutation
c3 = mutate_solution_1(c1)
c4 = mutate_solution_1(c2)
print('mutation of child 1 -----> {}'.format(c3))
print('mutation of child 2 -----> {}'.format(c4))

solution 1 -----> [ 97 100  12  64   1  68  55  31  40  67  31   7  48  90  22  16  81  52
  51  55  76   5  71   2  94  38  98  91  29   8   2  43  73  45  22  15
   8  32  93  83  87  59  39   9 100  69  66  34  47  97  90  27  42 100
  18   6  43  20  30  58 100  30  16  87  13  81  32  52  53  70  32  11
  32  32   9  79   2  41  44  54  47  88  79  25  91  80  35  69  31  38
  35  61  72  74  45   7  85   7  36  96  24  87  87  98  64  38  53  76
  35  63  59  81  58  73  43  80  42  99  20  54  20  39  89   5  46   9
  55  42  32  57  87  68  63  37  23  90  13  15  60  18  20  53  68  30
  48  66   4  98  57  21  17  74  97  80  16  99  60  27  96  89  33  13
  14  28  50  10  14  47  75  37  84  38  88  44   7  31   8  40  49  62
  93  88  29  82   9  80   2  24  30  11  72   8 100  52  60  73  67  36
  47  91  67  16  15  83  24  51  56  29  61  72  23   6  31  85  86  28
  23  51  24  46 100  94  17  74  51  50  13  90  42  37  77  29  46  22
  49 100  96  52  59   6   9  50 

In [7]:
pop = generate_population(10)
print_population(pop, objective_function_a_star)

tournament = tournament_select(pop, 2, objective_function_a_star)
print('Tournament -----> {}'.format(tournament))

best_fit = get_greatest_fit(pop, objective_function_a_star)
print('Greatest fit -----> {}'.format(best_fit))

roulette = roulette_select(pop, objective_function_a_star)
print('Roulette -----> {}'.format(roulette))

Solution 1: [ 16  31  64 100  24  17  51  52  84  72  31  11  14   8  62  19  85  52
  65  16  10  98   3  18  43  12  33  70  11  66  59   9  88  91  80  67
  98  61  89  30  12  95  53  37  92  52  84  64  13  23  48  50  22  95
 100  28  51  23  95  71  94  73  42   1  65 100  48  76  29  30  19  98
  29   6  17  82  19  20  91  58  76   5  60  15  32  50  27  78  40  41
  60  41  56  55  35  62   2  53  54   6  56  39  70  58  26  11  29  74
  30  56  61  14  25  68  74  37   9  62  15   8  44   6  28  32  32  79
   1  21  56  73  98  77  58  57  52  13  19   3  46  13  80  27  51  97
  92  22  51  41  79  99  85  75  18  75  15  14  21  50  32  33  85  67
  74  60  46  70   4  70  44 100  22  70   6  70  61  15   5  86  67  17
  30  65  49  14  89  82   1  94   5  49  93  74  72  17  67  59  50  49
  18  92  23  19  89  60  69  42  89  43  86  85  48  67  74  22  29  16
 100  84  16  85  83  63  58  81  52  65  57  83   2  34  80  59  11   1
  45  91  34  13  25   3  32  49  20  5

Solution 4: [ 38  98  24  87  99  99  14  14  22  45   9  45  67  39  89  69  43  44
  46  91  59  48  45  61 100  35  36  59  91  30  97  98  97  68  87  75
  49  63  52  32  97  76  23  67  94  30  68  84  14  28  83   4  84  30
  44   5  36  38  88  36  46  67  40  45  44  83  69  89  73  71  98  32
   1   8  82  17  65  86  40  40  47  62  79  47  75  88  92  38   2  17
  81  76  98  45  25  14  94  74  64  36  59  36   7   1  79   8  61  89
  19  13  87  34  63  95  33   1  83  78  58  56  62   8  44  23  66  43
  21  26  42  81  42  37  17  52  74  61  67  90  44  19  99  21  24  38
  97   7  22  59  77  61  12  61  95   9  21  27  48  95   8  65  95  81
  18   2  40  86  91  96  86  58  77  56  37  17  87  42  76  86  21   8
  15  15  20   9  86  36  54  62  37  10  13  63  49  57  68  25  78  52
  15  63   5  29   1  52  60  22  72  68  12   8  67  83  93  86  77  45
  34  97  32  40   3  80  90  28  51  92  74  57  40  14  68  67  97  76
  76  31   1  43  33  98  88  97   4  8

Solution 7: [ 71   8  75  84  75  50  36  43   5  43  56  49   2  21  25  82  52   9
   6  97   2  63  39  83  49  99  17  35  26  77  11  37  48  30  27  32
  38  86  88  41  51  12  70  83  53  58  26  52  29  44   2  93  37  23
  91  47  46  63 100   9  74  45  74  61  33  47  13  41   2  83  95  94
  64  43  21  96  70  95  75  99  96  70  30  80  67  21  75  18  75  53
  45   8   8   3  57   2  85  64  65  90  41  55  29  28  74  75  82  13
  22  93  59  42  48   4  60  96  93  27  17  32   8  18   1   9  64  28
  16  38  85  30  68  72  65  40  64  30  37  56  89  22   7  74  80  71
  22  43  97  66  82  46  21   2  11  74  45  75   4  63  58  18  91  80
  28   4  51  30  17  29  64  25  31  94  14  22  34  77  13  14  67  65
  76  44  42  23  14  51  70  90  69  27  46  85  40   1  85  76  80  23
  37   7  64  74  76   4  21  51  73  30  21  49  38  76  37  97  17  80
  37   3  81  81  25  94   1  27  22   3  39  10  54  58  42  48  61  38
  40   7  91  26  90  41  58  32  96  6

Solution 10: [ 17  70  83  78  24  41   4  25  82  94  93  50  10  43  24  46  64  49
 100  80  51  29  28  30  75  76  77  38  55  60  40  63  51  54  74   1
  74  80  95  59  45  81  26  91  69  36  91   3  62  21  10   7  75  16
  64  87  63  23  30  90  80  19  10  19   4  64  36  61  27  40  58  30
   2  23  20  70  24  77  91  92  33  46  55  37   2  18  90   1   8  72
  55  66  81  20  12  29  33   3  12  52   4  37  68  66  92  34  84  77
  78  77  24   6  98  95  96  68  44  95  88  34  25  90  16  52  94  90
  23  20   3 100  94  37  24  86  43  37  39  40  59  86  24   3  76  68
  74  38  34  33  24  97  51  18  53  17  11  16  16  74  43   5  19  71
  26   5   4  80  12  41  92  66  68  61  68  52  84  38  12  20  59  64
  35  57  67  43  86  32   8  52  11  31   7  69  30  39  54  90  44  15
  41  18  70  63  68  54  70  63  90  15  75  61  87  86  59  98  96  43
  53  58  13  74  60 100  80  77  69  52  10  44  41  46  30   4  37  50
   9 100  63  63  97   9  43  62  83  

Roulette -----> [ 16  31  64 100  24  17  51  52  84  72  31  11  14   8  62  19  85  52
  65  16  10  98   3  18  43  12  33  70  11  66  59   9  88  91  80  67
  98  61  89  30  12  95  53  37  92  52  84  64  13  23  48  50  22  95
 100  28  51  23  95  71  94  73  42   1  65 100  48  76  29  30  19  98
  29   6  17  82  19  20  91  58  76   5  60  15  32  50  27  78  40  41
  60  41  56  55  35  62   2  53  54   6  56  39  70  58  26  11  29  74
  30  56  61  14  25  68  74  37   9  62  15   8  44   6  28  32  32  79
   1  21  56  73  98  77  58  57  52  13  19   3  46  13  80  27  51  97
  92  22  51  41  79  99  85  75  18  75  15  14  21  50  32  33  85  67
  74  60  46  70   4  70  44 100  22  70   6  70  61  15   5  86  67  17
  30  65  49  14  89  82   1  94   5  49  93  74  72  17  67  59  50  49
  18  92  23  19  89  60  69  42  89  43  86  85  48  67  74  22  29  16
 100  84  16  85  83  63  58  81  52  65  57  83   2  34  80  59  11   1
  45  91  34  13  25   3  32  49  2

In [8]:
def genetic_algorithm(num_iterations, population_size, crossover_func, mutation_func, objective_func, log=False):
    population = generate_population(population_size)
    
    best_solution = population[0] # Initial solution
    best_score = objective_func(population[0], traveling_times, inspection_times, open_hours)[0]
    best_solution_generation = 0 # Generation on which the best solution was found
    
    generation_no = 0
    
    print(f"Initial solution: {best_solution}, score: {-best_score}")
    
    while(num_iterations > 0):
        
        generation_no += 1
        
        tournment_winner_sol = tournament_select(population, 4, objective_func)
        roulette_winner_sol = roulette_select(population, objective_func)
        
        # Next generation
        crossover_sol_1, crossover_sol_2 = crossover_func(tournment_winner_sol, roulette_winner_sol)
    
        if np.random.randint(0, 10) < 1: # 40% chance to perform mutation
            replace_least_fittest(population, mutation_func(crossover_sol_1), objective_func)
            replace_least_fittest(population, mutation_func(crossover_sol_2), objective_func)
        else:
            replace_least_fittest(population, crossover_sol_1, objective_func)
            replace_least_fittest(population, crossover_sol_2, objective_func)
        
        # Checking the greatest fit among the current population
        greatest_fit, greatest_fit_score, best_routes = get_greatest_fit(population, objective_func)
        if greatest_fit_score > best_score:
            best_solution = greatest_fit
            best_score = greatest_fit_score
            best_routes
            best_solution_generation = generation_no
            if log:
                print(f"\nGeneration: {generation_no }")
                print(f"Solution: score: {-best_score}")
                print_population(population, objective_func)
        else:
            num_iterations -= 1
        
    print(f"  Final solution: {best_solution}, score: {-best_score}, best routes: {best_routes}")
    print(f"  Found on generation {best_solution_generation}")
    
    return best_solution, best_routes

In [54]:
print("\nGeneticAlgorithm:\n")
solution, routes = genetic_algorithm(200, 30, randompoint_crossover, mutate_solution_1, objective_function_a_star, True)


GeneticAlgorithm:

Initial solution: [1 1 1 1 1 2 1 2 1 2 1 2 2 2 1 2 1 2 2 1], score: 159657.90000000002

Generation: 1
Solution: score: 120357.70000000001
Solution 1: [1 1 1 1 1 2 1 2 1 2 1 2 2 2 1 2 1 2 2 1], 159657.90000000002
Solution 2: [2 1 1 1 2 2 2 1 2 2 2 1 2 1 2 2 2 1 2 1], 146146.9
Solution 3: [1 1 1 1 1 2 2 2 1 1 2 2 2 2 1 1 1 2 1 1], 149415.1
Solution 4: [2 2 2 2 2 2 1 2 1 2 2 2 2 2 1 2 2 2 1 1], 158439.6
Solution 5: [1 2 1 2 2 1 2 2 1 2 2 2 2 2 2 1 2 1 1 2], 146388.2
Solution 6: [1 1 1 1 1 1 1 2 1 2 2 2 1 2 1 1 2 1 1 1], 128888.0
Solution 7: [2 1 1 1 1 2 1 2 1 1 1 2 2 1 2 1 2 2 1 1], 143066.2
Solution 8: [1 2 2 2 2 2 1 2 2 1 2 1 1 2 2 2 2 1 2 2], 120357.70000000001
Solution 9: [2 1 1 2 1 1 2 2 2 1 1 1 2 2 2 2 1 2 2 1], 170491.8
Solution 10: [1 1 1 1 1 1 1 2 1 2 1 1 1 2 1 1 1 1 2 1], 126709.6
Solution 11: [1 2 2 1 2 2 2 1 1 2 2 1 1 2 1 1 2 1 2 2], 153945.80000000002
Solution 12: [1 2 1 2 1 2 2 2 1 2 2 2 1 2 1 1 2 1 1 1], 135935.6
Solution 13: [1 1 2 2 2 2 1 1 1 1 1 2 1 2

  Final solution: [1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 1 1 1 1 1], score: 111212.80000000002, best routes: [[0, 19, 17, 5, 2, 6, 16, 4, 1, 14, 20, 7, 18, 3, 9, 0], [0, 13, 11, 15, 10, 12, 8, 0]]
  Found on generation 42


In [53]:
print("\nGeneticAlgorithm:\n")
solution, routes = genetic_algorithm(200, 30, uniform_crossover, mutate_solution_1, objective_function, True)


GeneticAlgorithm:

Initial solution: [2 1 2 2 1 2 2 2 2 1 2 1 2 1 2 2 2 2 1 1], score: 256233.0

Generation: 1
Solution: score: 179564.1
Solution 1: [2 1 2 2 1 2 2 2 2 1 2 1 2 1 2 2 2 2 1 1], 256233.0
Solution 2: [2 2 2 1 2 2 1 1 1 1 1 1 1 2 2 2 1 1 1 2], 234915.3
Solution 3: [2 2 2 2 2 2 2 1 1 2 1 2 1 1 2 2 1 1 2 1], 198805.8
Solution 4: [2 1 2 1 2 2 2 2 2 1 1 2 2 2 1 2 1 1 1 1], 240792.2
Solution 5: [1 2 2 1 2 1 1 2 2 2 1 1 1 2 1 2 2 2 2 1], 238063.1
Solution 6: [1 1 2 2 2 2 2 2 1 2 2 1 1 2 2 1 2 2 2 2], 229648.30000000002
Solution 7: [2 2 1 1 1 1 2 2 2 2 2 2 2 2 1 1 2 2 2 1], 192795.39999999997
Solution 8: [2 2 2 1 1 2 1 2 1 2 2 1 2 2 2 1 1 2 1 1], 188525.90000000002
Solution 9: [2 2 2 1 2 1 2 2 2 2 2 2 2 2 2 2 2 2 1 1], 251684.19999999998
Solution 10: [1 2 2 2 1 2 2 1 1 2 1 1 2 1 1 2 2 1 2 1], 240065.40000000002
Solution 11: [1 1 1 2 1 2 2 2 2 1 1 2 2 2 1 2 2 1 2 2], 236872.60000000003
Solution 12: [1 1 2 1 2 1 2 2 2 1 2 2 2 2 2 1 2 1 1 1], 262879.0
Solution 13: [1 1 2 2 2 2 1 1 2

In [55]:
best_greedy = [1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1]
best_a_star = [1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1]
score, routes = objective_function(best_greedy, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 5, 12, 6, 1, 3, 14, 7, 20, 15, 9, 0], [0, 17, 18, 8, 16, 2, 13, 4, 11, 10, 0]]
Objective function's values ---> 175714.7


In [56]:
best_greedy = [1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1]
best_a_star = [1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1]
score, routes = objective_function_a_star(best_a_star, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 17, 5, 2, 6, 16, 4, 1, 14, 20, 7, 18, 3, 9, 0], [0, 13, 11, 15, 10, 12, 8, 0]]
Objective function's values ---> 111212.80000000002


# Simulated Annealing

In [9]:
# Neighborhood definition

def get_neighbor_solution1(solution):
    neighbor = copy.deepcopy(solution)
    establishment_to_mutate = np.random.randint(0, num_establishments)
    neighbor[establishment_to_mutate] = (neighbor[establishment_to_mutate] + np.random.randint(1, vehicles)-1) % vehicles + 1
    return neighbor

# Exchange the vehicles of two establishments
def get_neighbor_solution2(solution):
    neighbor_solution = copy.deepcopy(solution)
    establishment1 = np.random.randint(0, num_establishments)
    establishment2 = (establishment1 + np.random.randint(1, num_establishments)) % num_establishments
    neighbor_solution[establishment1], neighbor_solution[establishment2] = neighbor_solution[establishment2], neighbor_solution[establishment1]
    return neighbor_solution

# Neighbour 1 or 2 with 50% each
def get_neighbor_solution3(solution):
    if (np.random.randint(0,2)==0):
        return get_neighbor_solution1(solution)
    else:
        return get_neighbor_solution2(solution)

In [10]:
#Test Neighbours
s = generate_random_solution()
print(s)
for d1 in range(0,20):
    print('Neighbor number {} with get_neighbor_solution1 ---> {}'.format(d1+1, get_neighbor_solution1(s)))

for d1 in range(0,20):
    print('Neighbor number {} with get_neighbor_solution2 ---> {}'.format(d1+1, get_neighbor_solution2(s)))

[ 98  49  22  31  72  79  78  35  54  24  80  37  32   3   1  76  64  81
  63  74   5  74  33  72  27  99   4  85   4  50  62  81  26  97  59  40
  37  78  80  14  80  31  18  91  98  85  83  40  75  75  81  58  39  83
  83  62  79  17  90  26  84  63  35  60  98   5  67  33  85   1  83  63
  38  69  80  17  44  56  54  50  70  35  58  87  47  40  62  26  10  13
  20  20  58  16  87  57  67  34  34  97  83 100  49  80  16  28  99  56
   1  29  57  54  13  41  64  60  47  41  76  70  31  99  51  93  94  52
  85  10  76   9  27  85  30  72  76  58  41  97  82  26  93  31  68  91
  27  76  81  87  29  38  52  43  11  14  34  31  37  27  88  40  25  77
   6  11  64  30  52  54  37  60  67  20  11  97  57  94  19  73  61  83
  29  87  11  38  47  74  66  12  85  64  21  20  62  17  94  12  97  89
  94  79  90  68  23  51  92  66  90  28  16  23  18  15 100  10  65  57
  82  86  41  84  24  18  24  97  55  12  59  94   8  53  47  62  22  92
  85  27  56  30  30  59  11   1  92  11  71   1  6

In [16]:
def get_sa_solution(num_iterations, neighbor_operator, traveling_times, inspection_times, open_hours, objective_func,log=False):
    iteration = 0;
    temperature = 1000;
    solution = generate_random_solution() # Best solution after 'num_iterations' iterations without improvement
    score, routes = objective_func(solution, traveling_times, inspection_times, open_hours)
    
    best_solution = copy.deepcopy(solution)
    best_score = score
    best_routes = routes
    
    print(f"Init Solution:  {best_solution}, score: {-best_score}")
    
    while iteration < num_iterations:
        temperature = temperature * 0.9999  # Test with different cooling schedules
        iteration += 1
        neighbor_solution = neighbor_operator(best_solution)  #Test with Neighbour 1, 2 and 3
        neighbor_score, neighbor_routes = objective_func(neighbor_solution, traveling_times, inspection_times, open_hours)
        
        delta = neighbor_score - score 
        
        if delta > 0 or np.exp(-delta/temperature) > random.random():
            solution = neighbor_solution
            score = neighbor_score
            routes = neighbor_routes
            if score > best_score:
                iteration = 0
                best_solution = copy.deepcopy(solution)
                best_score = score
                best_routes = routes
                if log:
                    print(f"Solution: score: {-best_score},  Temp: {temperature}")
                
    print(f"Final Solution: {best_solution}, score: {-best_score}")
    return best_solution, best_routes

In [62]:
print("\nSimulated Annealing:\n")
solution, routes = get_sa_solution(2000, get_neighbor_solution1, traveling_times, inspection_times, open_hours, objective_function, True)


Simulated Annealing:

Init Solution:  [2 2 2 1 1 2 2 2 1 1 1 1 1 1 1 1 2 1 2 2], score: 211397.6
Solution: score: 183625.0,  Temp: 969.4605362958226
Solution: score: 180350.5,  Temp: 962.6946373158061
Solution: score: 180166.5,  Temp: 958.8496310845508
Solution: score: 160361.6,  Temp: 955.0199818235594
Final Solution: [2 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 2 1 2 2], score: 160361.6


In [17]:
print("\nSimulated Annealing:\n")
solution, routes = get_sa_solution(2000, get_neighbor_solution1, traveling_times, inspection_times, open_hours, objective_function_a_star, True)


Simulated Annealing:

Init Solution:  [ 12  57  44  65  87  37  44  34  58  44 100  19  29  62  21  62  61  74
  66  94  97  29  36  50  87  88  42  80  45  22  98  66  94   8  23  61
  57  40  78  64  88  94  90  99  83 100  97  80  30  20  22   3  63  26
   7  41  38  13  34  67  45  13  49  85   9  79   6  14  90  40  99  82
   9  16  93  97  49  13  10  15  45  24  30   9  88  59  35  33  84   3
  39   6  75  13  35  60  41  98  23  88  26  85  79  81  13  62   3  66
  95  16   7  92  46  63  97  29  95  92  54  59  86  15  77  90  41  31
  51  75  62  62  20  72  72  14  47   3  94  19  32  61  94  67  11  19
   3  46  60  69  30   5  46   7  94  21  67  55  87  92  67  99  10  59
  60  39  82  48  13  23  41  53  83  86  69  61  63  22  46  81  57  25
  26  41  77  98  52   1  73  25  31  41  45  23  99   7  68  93  51  70
  65  21  79   2  17  53  76  86  70  91   5  72  81  48  78  64  49  40
  18  51  15  28  45  72  26  29  86  70  17  12  52  61  84  46  25  25
  46  91  47

KeyboardInterrupt: 

In [64]:
best_greedy = [2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2]
best_a_star = [2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1]
score, routes = objective_function(best_greedy, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 5, 18, 12, 16, 2, 3, 13, 14, 4, 11, 10, 15, 9, 0], [0, 19, 17, 8, 6, 1, 7, 20, 0]]
Objective function's values ---> 160361.6


In [65]:
best_greedy = [2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2]
best_a_star = [2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1]
score, routes = objective_function_a_star(best_a_star, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 17, 5, 6, 14, 11, 20, 7, 12, 8, 18, 3, 0], [0, 16, 2, 1, 13, 4, 15, 10, 9, 0]]
Objective function's values ---> 108482.90000000001


# Tabu Search

In [12]:
def get_tabu_search_solution(num_iterations, tabu_list_length, neighbor_operator, traveling_times, inspection_times, open_hours, objective_func,log=False):
    current_solution = generate_random_solution()
    current_score, current_route = objective_func(current_solution, traveling_times, inspection_times, open_hours)
    
    best_solution = current_solution
    best_score = current_score
    best_route = current_route
    
    tabu_list = []
    
    for iteration in range(num_iterations):
        neighbor_solutions = []
        neighbor_scores = []
        neighbor_routes = []
        tabu_neighbors = []
        
        for i in range(10):
            neighbor_solution = neighbor_operator(current_solution)
            neighbor_score, neighbor_route = objective_func(neighbor_solution, traveling_times, inspection_times, open_hours)
            if neighbor_solution.tolist() not in tabu_list:
                neighbor_solutions.append(neighbor_solution)
                neighbor_scores.append(neighbor_score)
                neighbor_routes.append(neighbor_route)
            else:
                tabu_neighbors.append(neighbor_solution)
        
        if len(neighbor_scores) > 0:
            best_neighbor_index = np.argmax(neighbor_scores)
            best_neighbor_score = neighbor_scores[best_neighbor_index]
            best_neighbor_solution = neighbor_solutions[best_neighbor_index]
            best_neighbor_route = neighbor_routes[best_neighbor_index]
            
            if best_neighbor_score > best_score:
                best_solution = best_neighbor_solution
                best_score = best_neighbor_score
                best_route = best_neighbor_route
            
            current_solution = best_neighbor_solution
            current_score = best_neighbor_score
            current_route = best_neighbor_route
            
            tabu_list.append(current_solution.tolist())
            if len(tabu_list) > tabu_list_length:
                tabu_list.pop(0)
                
        elif len(tabu_neighbors) > 0:
            best_tabu_index = np.argmax([objective_func(x, traveling_times, inspection_times, open_hours)[0] for x in tabu_neighbors])
            best_tabu_solution = tabu_neighbors[best_tabu_index]
            
            current_solution = best_tabu_solution
            current_score, current_route = objective_func(current_solution, traveling_times, inspection_times, open_hours)
            
            tabu_list.append(current_solution.tolist())
            if len(tabu_list) > tabu_list_length:
                tabu_list.pop(0)
        
        if log:
            print(f"Iteration {iteration}, Best Score: {-best_score}")
    
    print(f"Best solution: {best_solution}, Best Score: {-best_score}, Routes: {best_route}")
    
    return best_solution, best_route

In [14]:
print("\nTabu Search:\n")
solution, routes = get_tabu_search_solution(500000, 50000, get_neighbor_solution3, traveling_times, inspection_times, open_hours, objective_function, True)


Tabu Search:

Iteration 0, Best Score: 11274137.400000008
Iteration 1, Best Score: 11245337.400000008
Iteration 2, Best Score: 11175810.800000006
Iteration 3, Best Score: 11134000.400000004


KeyboardInterrupt: 

In [ ]:
print("\nTabu Search:\n")
solution, routes = get_tabu_search_solution(20000, 2000, get_neighbor_solution3, traveling_times, inspection_times, open_hours, objective_function_a_star, True)


Tabu Search:

Iteration 0, Best Score: 7864987.900000001
Iteration 1, Best Score: 7803940.1000000015
Iteration 2, Best Score: 7751528.6000000015
Iteration 3, Best Score: 7704340.6000000015
Iteration 4, Best Score: 7647291.000000001
Iteration 5, Best Score: 7642905.500000001
Iteration 6, Best Score: 7583880.4
Iteration 7, Best Score: 7560507.5
Iteration 8, Best Score: 7555944.9
Iteration 9, Best Score: 7541979.2
Iteration 10, Best Score: 7500807.8
Iteration 11, Best Score: 7462448.899999999
Iteration 12, Best Score: 7452113.699999999
Iteration 13, Best Score: 7439900.399999999
Iteration 14, Best Score: 7379922.1
Iteration 15, Best Score: 7360302.2
Iteration 16, Best Score: 7341724.4
Iteration 17, Best Score: 7299502.000000001
Iteration 18, Best Score: 7292165.100000001
Iteration 19, Best Score: 7288867.100000001
Iteration 20, Best Score: 7287088.300000001
Iteration 21, Best Score: 7277797.300000001
Iteration 22, Best Score: 7264997.000000001
Iteration 23, Best Score: 7222011.100000001


Iteration 188, Best Score: 6131773.500000003
Iteration 189, Best Score: 6131773.500000003
Iteration 190, Best Score: 6124874.800000003
Iteration 191, Best Score: 6122548.000000002
Iteration 192, Best Score: 6119842.900000002
Iteration 193, Best Score: 6116620.800000002
Iteration 194, Best Score: 6115950.000000002
Iteration 195, Best Score: 6114768.300000001
Iteration 196, Best Score: 6114768.300000001
Iteration 197, Best Score: 6113041.4
Iteration 198, Best Score: 6108492.300000001
Iteration 199, Best Score: 6108492.300000001
Iteration 200, Best Score: 6108492.300000001
Iteration 201, Best Score: 6106643.300000001
Iteration 202, Best Score: 6104881.400000001
Iteration 203, Best Score: 6104881.400000001
Iteration 204, Best Score: 6103877.200000001
Iteration 205, Best Score: 6100590.800000001
Iteration 206, Best Score: 6100061.900000001
Iteration 207, Best Score: 6097312.800000001
Iteration 208, Best Score: 6095225.300000001
Iteration 209, Best Score: 6094947.2
Iteration 210, Best Score:

Iteration 374, Best Score: 5841663.699999998
Iteration 375, Best Score: 5841663.699999998
Iteration 376, Best Score: 5841663.699999998
Iteration 377, Best Score: 5841663.699999998
Iteration 378, Best Score: 5841663.699999998
Iteration 379, Best Score: 5841663.699999998
Iteration 380, Best Score: 5841663.699999998
Iteration 381, Best Score: 5841663.699999998
Iteration 382, Best Score: 5841663.699999998
Iteration 383, Best Score: 5841663.699999998
Iteration 384, Best Score: 5841663.699999998
Iteration 385, Best Score: 5841663.699999998
Iteration 386, Best Score: 5839562.499999997
Iteration 387, Best Score: 5837156.499999997
Iteration 388, Best Score: 5837156.499999997
Iteration 389, Best Score: 5837156.499999997
Iteration 390, Best Score: 5837156.499999997
Iteration 391, Best Score: 5835616.499999998
Iteration 392, Best Score: 5835218.3999999985
Iteration 393, Best Score: 5835218.3999999985
Iteration 394, Best Score: 5835218.3999999985
Iteration 395, Best Score: 5835218.3999999985
Iterat

Iteration 564, Best Score: 5711953.199999998
Iteration 565, Best Score: 5711953.199999998
Iteration 566, Best Score: 5711953.199999998
Iteration 567, Best Score: 5711953.199999998
Iteration 568, Best Score: 5710800.499999999
Iteration 569, Best Score: 5708737.499999999
Iteration 570, Best Score: 5706973.999999999
Iteration 571, Best Score: 5705085.699999998
Iteration 572, Best Score: 5705085.699999998
Iteration 573, Best Score: 5705085.699999998
Iteration 574, Best Score: 5703292.8999999985
Iteration 575, Best Score: 5703292.8999999985
Iteration 576, Best Score: 5703292.8999999985
Iteration 577, Best Score: 5703292.8999999985
Iteration 578, Best Score: 5702613.199999998
Iteration 579, Best Score: 5701185.699999999
Iteration 580, Best Score: 5700333.8999999985
Iteration 581, Best Score: 5700060.599999999
Iteration 582, Best Score: 5700060.599999999
Iteration 583, Best Score: 5700060.599999999
Iteration 584, Best Score: 5700060.599999999
Iteration 585, Best Score: 5699641.499999998
Itera

Iteration 748, Best Score: 5631024.000000002
Iteration 749, Best Score: 5631024.000000002
Iteration 750, Best Score: 5624793.300000004
Iteration 751, Best Score: 5624793.300000004
Iteration 752, Best Score: 5624793.300000004
Iteration 753, Best Score: 5624793.300000004
Iteration 754, Best Score: 5624793.300000004
Iteration 755, Best Score: 5624793.300000004
Iteration 756, Best Score: 5624265.200000002
Iteration 757, Best Score: 5624265.200000002
Iteration 758, Best Score: 5624265.200000002
Iteration 759, Best Score: 5624265.200000002
Iteration 760, Best Score: 5624265.200000002
Iteration 761, Best Score: 5621304.700000002
Iteration 762, Best Score: 5621304.700000002
Iteration 763, Best Score: 5621304.700000002
Iteration 764, Best Score: 5620896.300000002
Iteration 765, Best Score: 5620667.400000002
Iteration 766, Best Score: 5620667.400000002
Iteration 767, Best Score: 5620667.400000002
Iteration 768, Best Score: 5616634.100000002
Iteration 769, Best Score: 5615704.700000001
Iteration 

...

Iteration 2412, Best Score: 5359831.300000001
Iteration 2413, Best Score: 5359831.300000001
Iteration 2414, Best Score: 5359831.300000001
Iteration 2415, Best Score: 5359831.300000001
Iteration 2416, Best Score: 5359831.300000001
Iteration 2417, Best Score: 5359831.300000001
Iteration 2418, Best Score: 5359831.300000001
Iteration 2419, Best Score: 5359831.300000001
Iteration 2420, Best Score: 5359831.300000001
Iteration 2421, Best Score: 5359831.300000001
Iteration 2422, Best Score: 5359831.300000001
Iteration 2423, Best Score: 5359831.300000001
Iteration 2424, Best Score: 5359831.300000001
Iteration 2425, Best Score: 5359831.300000001
Iteration 2426, Best Score: 5359831.300000001
Iteration 2427, Best Score: 5359831.300000001
Iteration 2428, Best Score: 5359831.300000001
Iteration 2429, Best Score: 5359831.300000001
Iteration 2430, Best Score: 5359831.300000001
Iteration 2431, Best Score: 5359831.300000001
Iteration 2432, Best Score: 5359831.300000001
Iteration 2433, Best Score: 535983

Iteration 2591, Best Score: 5359831.300000001
Iteration 2592, Best Score: 5359831.300000001
Iteration 2593, Best Score: 5359831.300000001
Iteration 2594, Best Score: 5359831.300000001
Iteration 2595, Best Score: 5359831.300000001
Iteration 2596, Best Score: 5359831.300000001
Iteration 2597, Best Score: 5359831.300000001
Iteration 2598, Best Score: 5359831.300000001
Iteration 2599, Best Score: 5359831.300000001
Iteration 2600, Best Score: 5359831.300000001
Iteration 2601, Best Score: 5359831.300000001
Iteration 2602, Best Score: 5359831.300000001
Iteration 2603, Best Score: 5359831.300000001
Iteration 2604, Best Score: 5359831.300000001
Iteration 2605, Best Score: 5359831.300000001
Iteration 2606, Best Score: 5359831.300000001
Iteration 2607, Best Score: 5359831.300000001
Iteration 2608, Best Score: 5359831.300000001
Iteration 2609, Best Score: 5359831.300000001
Iteration 2610, Best Score: 5359831.300000001
Iteration 2611, Best Score: 5359831.300000001
Iteration 2612, Best Score: 535983

Iteration 2770, Best Score: 5359831.300000001
Iteration 2771, Best Score: 5359831.300000001
Iteration 2772, Best Score: 5359831.300000001
Iteration 2773, Best Score: 5359831.300000001
Iteration 2774, Best Score: 5359831.300000001
Iteration 2775, Best Score: 5359831.300000001
Iteration 2776, Best Score: 5359831.300000001
Iteration 2777, Best Score: 5359831.300000001
Iteration 2778, Best Score: 5359831.300000001
Iteration 2779, Best Score: 5359831.300000001
Iteration 2780, Best Score: 5359831.300000001
Iteration 2781, Best Score: 5359831.300000001
Iteration 2782, Best Score: 5359831.300000001
Iteration 2783, Best Score: 5359831.300000001
Iteration 2784, Best Score: 5359831.300000001
Iteration 2785, Best Score: 5359831.300000001
Iteration 2786, Best Score: 5359831.300000001
Iteration 2787, Best Score: 5359831.300000001
Iteration 2788, Best Score: 5359831.300000001
Iteration 2789, Best Score: 5359831.300000001
Iteration 2790, Best Score: 5359831.300000001
Iteration 2791, Best Score: 535983

Iteration 2949, Best Score: 5359831.300000001
Iteration 2950, Best Score: 5359831.300000001
Iteration 2951, Best Score: 5359831.300000001
Iteration 2952, Best Score: 5359831.300000001
Iteration 2953, Best Score: 5359831.300000001
Iteration 2954, Best Score: 5359831.300000001
Iteration 2955, Best Score: 5359831.300000001
Iteration 2956, Best Score: 5359831.300000001
Iteration 2957, Best Score: 5359831.300000001
Iteration 2958, Best Score: 5359831.300000001
Iteration 2959, Best Score: 5359831.300000001
Iteration 2960, Best Score: 5359831.300000001
Iteration 2961, Best Score: 5359831.300000001
Iteration 2962, Best Score: 5359831.300000001
Iteration 2963, Best Score: 5359831.300000001
Iteration 2964, Best Score: 5359831.300000001
Iteration 2965, Best Score: 5359831.300000001
Iteration 2966, Best Score: 5359831.300000001
Iteration 2967, Best Score: 5359831.300000001
Iteration 2968, Best Score: 5359831.300000001
Iteration 2969, Best Score: 5359831.300000001
Iteration 2970, Best Score: 535983

Iteration 3128, Best Score: 5359831.300000001
Iteration 3129, Best Score: 5359831.300000001
Iteration 3130, Best Score: 5359831.300000001
Iteration 3131, Best Score: 5359831.300000001
Iteration 3132, Best Score: 5359831.300000001
Iteration 3133, Best Score: 5359831.300000001
Iteration 3134, Best Score: 5359831.300000001
Iteration 3135, Best Score: 5359831.300000001
Iteration 3136, Best Score: 5359831.300000001
Iteration 3137, Best Score: 5359831.300000001
Iteration 3138, Best Score: 5359831.300000001
Iteration 3139, Best Score: 5359831.300000001
Iteration 3140, Best Score: 5359831.300000001
Iteration 3141, Best Score: 5359831.300000001
Iteration 3142, Best Score: 5359831.300000001
Iteration 3143, Best Score: 5359831.300000001
Iteration 3144, Best Score: 5359831.300000001
Iteration 3145, Best Score: 5359831.300000001
Iteration 3146, Best Score: 5359831.300000001
Iteration 3147, Best Score: 5359831.300000001
Iteration 3148, Best Score: 5359831.300000001
Iteration 3149, Best Score: 535983

In [91]:
best_greedy = [2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2]
best_a_star = [1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 2]
score, routes = objective_function(best_greedy, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 17, 18, 8, 2, 3, 14, 4, 11, 10, 15, 9, 0], [0, 19, 5, 12, 6, 16, 1, 13, 7, 20, 0]]
Objective function's values ---> 123019.20000000001


In [81]:
best_greedy = [2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2]
best_a_star = [1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 2]
score, routes = objective_function_a_star(best_a_star, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 17, 8, 2, 14, 1, 3, 13, 4, 11, 15, 10, 9, 0], [0, 19, 5, 6, 16, 20, 7, 12, 18, 0]]
Objective function's values ---> 106603.6


# Hill-Climbing

In [84]:
def get_hc_solution(num_iterations, neighbor_operator, traveling_times, inspection_times, open_hours, objective_func, log=False):
    iteration = 0;
    best_solution = generate_random_solution() # Best solution after 'num_iterations' iterations without improvement
    best_score, best_routes = objective_func(best_solution, traveling_times, inspection_times, open_hours)
    
    print(f"Init Solution: score: {-best_score}")
    
    while iteration < num_iterations:
        iteration += 1
        neighbor_solution = neighbor_operator(best_solution)   #Test with Neighbour 1, 2 and 3
        neighbor_score, neighbor_routes = objective_func(neighbor_solution, traveling_times, inspection_times, open_hours)
        if neighbor_score > best_score:
            iteration = 0
            best_solution = neighbor_solution
            best_score = neighbor_score
            best_routes = neighbor_routes
            if log:
                (print(f"Solution {iteration}: score: {-best_score}"))
            
    print(f"Final Solution: solution: {best_solution}, score: {-best_score}, routes: {best_routes}")
    return best_solution, best_routes

In [86]:
print("Hill climbing:\n")
solution, routes = get_hc_solution(2000, get_neighbor_solution3, traveling_times, inspection_times, open_hours, objective_function, True)

Hill climbing:

Init Solution: score: 174179.40000000002
Solution 0: score: 173476.1
Solution 0: score: 155452.5
Solution 0: score: 153530.90000000002
Solution 0: score: 149474.5
Solution 0: score: 128726.6
Final Solution: solution: [1 2 1 2 1 1 1 2 2 2 2 1 1 1 2 1 1 2 1 2], score: 128726.6, routes: [[0, 19, 17, 5, 12, 6, 16, 1, 3, 13, 14, 7, 0], [0, 18, 8, 2, 4, 11, 20, 10, 15, 9, 0]]


In [87]:
print("Hill climbing:\n")
solution, routes = get_hc_solution(2000, get_neighbor_solution3, traveling_times, inspection_times, open_hours, objective_function_a_star, True)

Hill climbing:

Init Solution: score: 176134.6
Solution 0: score: 148627.40000000002
Solution 0: score: 123451.3
Solution 0: score: 120608.30000000002
Solution 0: score: 118596.20000000001
Solution 0: score: 114514.9
Solution 0: score: 114105.2
Solution 0: score: 112161.5
Solution 0: score: 111907.0
Solution 0: score: 108184.40000000001
Solution 0: score: 106920.00000000001
Solution 0: score: 106603.6
Final Solution: solution: [2 2 2 2 1 1 1 2 2 2 2 1 2 2 2 1 2 1 1 1], score: 106603.6, routes: [[0, 19, 5, 6, 16, 20, 7, 12, 18, 0], [0, 17, 8, 2, 14, 1, 3, 13, 4, 11, 15, 10, 9, 0]]


In [88]:
best_greedy = [1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2]
best_a_star = [2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1]
score, routes = objective_function(best_greedy, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 17, 5, 12, 6, 16, 1, 3, 13, 14, 7, 0], [0, 18, 8, 2, 4, 11, 20, 10, 15, 9, 0]]
Objective function's values ---> 128726.6


In [90]:
best_greedy = [1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2]
best_a_star = [2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1]
score, routes = objective_function_a_star(best_a_star, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 5, 6, 16, 20, 7, 12, 18, 0], [0, 17, 8, 2, 14, 1, 3, 13, 4, 11, 15, 10, 9, 0]]
Objective function's values ---> 106603.6


# Problem 2

In [ ]:
# convert the DataFrame to a dictionary
traveling_times = {}
for i in range(len(very_small_distances)):
    traveling_times[i] = very_small_distances.iloc[i].tolist()

num_establishments = len(traveling_times)-1
inspection_times = very_small_establishments[['Inspection Time']]
open_hours = very_small_establishments[['Opening Hours']]

def generate_random_solution():
    solution = np.zeros(num_establishments, dtype=int)
    values = list(range(1, vehicles + 1))
    np.random.shuffle(values)
    index = 0
    for i in range(num_establishments):
        solution[i] = values[index]
        index = (index + 1) % vehicles
    return solution


def objective_function2(chromosome, traveling_times, inspection_times, open_hours):
    total_travel_time = 0
    total_inspection_time = 0
    total_time_per_route = [0 for i in range(max(chromosome))]
    j = -1

    # Initialize an empty list to store the routes
    routes = [[] for i in range(max(chromosome))]

    # Assign each establishment to a route
    for i in range(len(chromosome)):
        routes[chromosome[i]-1].append(i+1)

    # Calculate the total travel time and inspection time for all routes
    for route in routes:
        if len(route) == 0:
            continue
        j += 1
        route_travel_time = 0
        route_inspection_time = 0
        current_time = 0
        # Add the departure/arrival establishment to the beginning and end of the route
        route = [0] + route
        
        # Use a greedy search algorithm to find the optimal path for the current route
        route = greedy_search(route, heuristic_distances)
        
        start_time = ast.literal_eval(open_hours.iloc[route[1]][0]).index(1)  # Inspection start time of first establishment in route
        #print('start time: {}'.format(start_time))

        for i in range(1, len(route)):
            # Calculate traveling time between establishments
            current_establishment = route[i-1]
            next_establishment = route[i]
            travel_time = traveling_times[current_establishment][next_establishment]
            #route_travel_time += traveling_times[current_establishment][next_establishment]
            
            # Calculate inspection time and waiting time based on establishment's schedule
            current_inspection_time = 5*60 # Convert minutes to seconds
            current_open_hours = ast.literal_eval(open_hours.iloc[next_establishment][0])
            
            # Calculate waiting time if establishment is closed
            if i == 1:
                waiting_time = 0
            else:
                current_hour = int((current_time + travel_time + (start_time*3600))/3600)
                #print('Current hour: {}'.format(current_hour))
                while current_open_hours[current_hour % 24] == 0:
                    current_hour += 1
                waiting_time = (current_hour * 3600) - (current_time + travel_time + (start_time*3600))
                #print('({} * 3600) - ({} + {} + ({}*3600))'.format(current_hour, current_time, travel_time, start_time))
                
            # Add inspection time and waiting time to current time
            if i > 1:
                current_time += max(waiting_time, 0)
            current_time += travel_time
            current_time += current_inspection_time
            
            if current_time > 28800:
                return -1000000000000
            
            #print('Iteration {} ---> current establishment: {}, next establishment: {}, inspection time: {}, open hours array: {}, travel time: {}, waiting time: {}, current time: {}'.format(i, current_establishment, next_establishment, current_inspection_time, current_open_hours, travel_time, waiting_time, current_time))
            # Reset waiting time
            waiting_time = 0
            
            # Add inspection time and traveling time to route times
            #route_inspection_time += current_inspection_time
            route_travel_time = 0

        # Add total time for current route to total time per route
        total_time_per_route[j] = current_time
        #print('total time per route ----> {}'.format(total_time_per_route))

    # Add total time for all routes to total travel time
    total_travel_time += sum(total_time_per_route)

def objective_function_a_star2(chromosome, traveling_times, inspection_times, open_hours):
    
    total_travel_time = 0
    l = 1
    
    # Initialize an empty list to store the routes
    routes = [[] for i in range(max(chromosome))]
    
    # Assign each establishment to a route
    for i in range(len(chromosome)):
        routes[chromosome[i]-1].append(i+1)
    
    # Calculate the total travel time for all routes
    for route in routes:
        if len(route) == 0:
            continue
        j = 1
        current_time = 0
        
        # Create a set with all establishments in the route
        route_establishments = set(route)
        route_len = len(route_establishments)
        
        # Add the depot to the beginning of the route
        route = [0]
        
        # Initialize the route travel time as zero
        route_travel_time = 0
        
        while route_establishments:
            # Calculate the A* algorithm for the next establishment
            current_establishment = route[-1]
            next_establishment = None
            best_score = float('inf')
            for e in route_establishments:
                i = 1
                # Calculate the total time (travel time, inspection time, waiting time, heuristic value) for the next establishment
                travel_time = traveling_times[route[-1]][e]
                inspection_time = 5 * 60
                current_open_hours = ast.literal_eval(open_hours.iloc[e][0])
                if len(route_establishments) == route_len:
                    start_time = current_open_hours.index(1)
                i += 1
                # Calculate waiting time if establishment is closed
                if len(route) == 1:
                    waiting_time = 0
                else:
                    current_hour = int((current_time + travel_time + (start_time*3600))/3600)
                    while current_open_hours[current_hour % 24] == 0:
                        current_hour += 1
                    waiting_time = (current_hour * 3600) - (current_time + travel_time + (start_time*3600))
                
                # Add inspection time, waiting time, and travel time to the current time
                time_to_next = max(waiting_time, 0) + inspection_time + travel_time
                total_time = current_time + time_to_next
                heuristic_value = heuristic_distances[e][0]
                score = total_time + heuristic_value
                
                # If the current establishment has the smallest score, update next_establishment
                if score < best_score:
                    '''
                    best_waiting_time = max(waiting_time, 0)
                    best_open_hours = current_open_hours
                    best_inspection_time = inspection_time
                    if len(route) == 1:
                        best_start_time = start_time
                    else:
                        best_current_hour = current_hour
                    best_current_time = total_time
                    best_travel_time = travel_time
                    '''
                    time_to_next_establishment = time_to_next
                    next_establishment = e
                    best_score = score
            '''
            if len(route) > 1:
                print('Current hour: {}'.format(best_current_hour))
                print('({} * 3600) - ({} + {} + ({}*3600))'.format(best_current_hour, current_time, best_travel_time, best_start_time))
            else:
                print('start time: {}'.format(best_start_time))
            '''
            
            # Add next establishment to route and remove from set
            route_establishments.remove(next_establishment)
            route.append(next_establishment)
            
            #print('Update current time: {} + {}'.format(current_time, time_to_next_establishment))
            # Update the current time and route travel time
            current_time += time_to_next_establishment
            #route_travel_time += travel_time
            #print('Iteration {} ---> current establishment: {}, next establishment: {}, next inspection time: {}, open hours array: {}, travel time: {}, waiting time: {}, current time: {}'.format(j, current_establishment, next_establishment, best_inspection_time, best_open_hours, best_travel_time, best_waiting_time, best_current_time))
        
            if current_time > 28800:
                return -1000000000000
        
            j += 1

        #print('Route: {}'.format(route))
        # Calculate the total time for the current route and add to total time per route
        total_time_per_route = current_time
        #print('Traveling time back to the depot: {}'.format(traveling_times[route[-2]][0]))
        #print('Total travel time in route {}: {}'.format(l, total_time_per_route))
        total_travel_time += total_time_per_route
        l += 1
        
    return -total_travel_time # Minimize the total travel time

In [ ]:
vehicles = 1

while True:
    print("\nSimulated Annealing:")
    print(f"Number of vehicles: {vehicles}\n")
    best_solution = get_sa_solution(5000, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function_a_star2,True)
    best_score = objective_function_a_star2(best_solution, traveling_times, inspection_times, open_hours)
    if best_score > -10000000:
        print(f"Final Solution: {best_solution}, score: {-best_score}, num_vehicles: {vehicles}")
        break
    vehicles += 1

In [ ]:
vehicles = 1

while True:
    print("\nTabu Search:")
    print(f"Number of vehicles: {vehicles}\n")
    best_solution = get_tabu_search_solution(500, 40, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function_a_star2, True)
    best_score = objective_function_a_star2(best_solution, traveling_times, inspection_times, open_hours)
    if best_score > -10000000:
        print(f"Final Solution: {best_solution}, score: {-best_score}, num_vehicles: {vehicles}")
        break
    vehicles += 1

In [ ]:
vehicles= 9

while True:
    vehicles += 1
    print("\nHill climbing:")
    print(f"Number of vehicles: {vehicles}\n")
    best_solution = get_hc_solution(1000, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function2, True)
    best_score = objective_function2(best_solution, traveling_times, inspection_times, open_hours)
    if best_score > -10000000:
        print(f"Final Solution: {best_solution}, score: {-best_score}, num_vehicles: {vehicles}")
        break